In [1]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [2]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# display dictionary
stateData


{'tennessee': {'white': {'population': 5200531,
   'inverted_ccvi': 45.91,
   'chance_of_infection': 15.44,
   'chance_of_death': 1.47,
   'generated_cases': 0,
   'generated_deaths': 0},
  'black': {'population': 1120221,
   'inverted_ccvi': 28.16,
   'chance_of_infection': 15.65,
   'chance_of_death': 1.68,
   'generated_cases': 0,
   'generated_deaths': 0},
  'native': {'population': 18151,
   'inverted_ccvi': 40.66,
   'chance_of_infection': 9.31,
   'chance_of_death': 1.12,
   'generated_cases': 0,
   'generated_deaths': 0},
  'asian': {'population': 117348,
   'inverted_ccvi': 49.7,
   'chance_of_infection': 9.41,
   'chance_of_death': 0.79,
   'generated_cases': 0,
   'generated_deaths': 0},
  'pacific': {'population': 3735,
   'inverted_ccvi': 37.37,
   'chance_of_infection': 24.52,
   'chance_of_death': 1.03,
   'generated_cases': 0,
   'generated_deaths': 0},
  'hispanic': {'population': 363753,
   'inverted_ccvi': 32.45,
   'chance_of_infection': 25.67,
   'chance_of_death':

In [3]:
# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [4]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
more = [0]
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 62, 63, 65, 67, 68, 68, 74, 77, 78, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': []
}


In [5]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,t):
    return 100 * n + t

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def thisThresh():
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    
# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([5,6,8])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([5,6,7,8])
        return random.randint(0, sub*40)*2000/sub
    
# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(t):
    note = random.choice(midiNotes)
    return note + t


In [6]:
while len(more) > 0:
    
    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        more.append(case)
        
#     # decide if asymptomatic or not
#     if decide(40) == 0:
        
    # add current case to total # of cases
    # (only sympotmatic cases adjust threshold)
    sympCount += 1

    # if case count is at or over threshold number
    if sympCount >= nthThresh(n, thresh):
        # calculate new threshold
        thresh = nthThresh(n, thresh)
        # adjust spreadProbs weights and calculate r0 for corresponding threshold
        thisThresh()
        # add one to threshold n value
        n += 1

    # select which demographic the current case belongs to
    demo = select(infectProbs)
    weathering['demo'].append(demo)

    # select which note
    weathering['note'].append(midi(0))

    # select incubation period (delay time)
    delay = incubate(1)

    # calculate time from beginning that symptoms occur for current case
    if sympCount == 1:
        time = delay
    else:
        time = delay + weathering['time'][more[0]]
    weathering['time'].append(time)

    # decide/record if case results in death or not
    death = decide(deathProbs[demo])
    weathering['death'].append(death)

    case += 1
        
    # remove current case from how many more cases left
    more.pop(0)
    


In [7]:
result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death']))

result


[(58000.0, 0, 77, 0),
 (98000.0, 3, 68, 1),
 (90666.66666666667, 0, 62, 0),
 (60000.0, 1, 78, 0),
 (122666.66666666667, 0, 74, 0),
 (112000.0, 1, 67, 0),
 (130666.66666666667, 3, 68, 0),
 (128666.66666666667, 1, 48, 0),
 (150666.6666666667, 2, 51, 0),
 (135333.33333333334, 0, 74, 0),
 (87333.33333333333, 5, 74, 0),
 (109333.33333333334, 2, 74, 0),
 (111333.33333333334, 0, 63, 0),
 (183333.33333333334, 2, 80, 0),
 (140000.0, 1, 48, 0),
 (184666.6666666667, 0, 62, 0),
 (171333.33333333334, 0, 77, 0),
 (162000.0, 1, 51, 0),
 (115333.33333333333, 2, 77, 0),
 (177333.33333333334, 0, 48, 0),
 (150000.0, 5, 63, 0),
 (190000.0, 5, 77, 0),
 (186666.6666666667, 5, 51, 0),
 (164000.0, 2, 63, 0),
 (174666.6666666667, 1, 58, 0),
 (200000.00000000003, 1, 63, 0),
 (198000.00000000003, 3, 68, 0),
 (228666.6666666667, 1, 48, 0),
 (162666.6666666667, 0, 67, 0),
 (183333.33333333334, 2, 77, 0),
 (191333.33333333334, 0, 63, 0),
 (157333.3333333333, 0, 78, 0),
 (103333.33333333333, 0, 74, 0),
 (122666.6666

In [8]:
result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death']))

result = sorted(result, reverse=False)

len(result)


11672

In [9]:
timing = {}
ms = 0
infections = -1

for time in result:

    if time[0] != ms:
        infections += 1
        timing[infections] = {'demoNotes': [time[1], time[2], time[3]]}
        timing[infections]['delay'] = time[0] - ms
        ms = time[0]

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(time[i])

#     if time[0] != ms:
#         timing[time[0]] = {'demoNotes': [time[1], time[2], time[3]]}
#         timing[time[0]]['delay'] = time[0] - ms
#         ms = time[0]

#     else:
#         for i in range(1,4):
#             timing[time[0]]['demoNotes'].append(time[i])


In [10]:
timing

{0: {'demoNotes': [0, 77, 0], 'delay': 58000.0},
 1: {'demoNotes': [1, 78, 0], 'delay': 2000.0},
 2: {'demoNotes': [5, 74, 0], 'delay': 27333.33333333333},
 3: {'demoNotes': [0, 62, 0], 'delay': 3333.333333333343},
 4: {'demoNotes': [3, 68, 1], 'delay': 7333.3333333333285},
 5: {'demoNotes': [0, 74, 0], 'delay': 5333.3333333333285},
 6: {'demoNotes': [2, 74, 0], 'delay': 6000.000000000015},
 7: {'demoNotes': [0, 63, 0], 'delay': 2000.0},
 8: {'demoNotes': [1, 67, 0], 'delay': 666.666666666657},
 9: {'demoNotes': [0, 68, 0], 'delay': 1333.333333333343},
 10: {'demoNotes': [2, 77, 0], 'delay': 1999.9999999999854},
 11: {'demoNotes': [1, 63, 0], 'delay': 2000.0000000000146},
 12: {'demoNotes': [0, 58, 0], 'delay': 5333.333333333314},
 13: {'demoNotes': [0, 74, 0], 'delay': 1.4551915228366852e-11},
 14: {'demoNotes': [1, 48, 0], 'delay': 6000.0},
 15: {'demoNotes': [3, 68, 0], 'delay': 2000.0},
 16: {'demoNotes': [5, 77, 0], 'delay': 666.6666666666424},
 17: {'demoNotes': [3, 48, 0], 'dela